# Near-horizon geometry of the extremal Kerr black hole


This notebook derives the near-horizon geometry of the extremal (i.e. maximally spinning) Kerr black hole. It is based on SageMath tools developed through the [SageManifolds project](https://sagemanifolds.obspm.fr/). Some of these tools, like the series expansion of tensor fields, are quite recent and require a version of SageMath at least equal to 8.8:

In [1]:
version()

'SageMath version 9.0.beta7, Release Date: 2019-11-26'

First we set up the notebook to display mathematical objects using LaTeX rendering:

In [2]:
%display latex

To speed up computations, we ask for running them in parallel on 8 threads:

In [3]:
Parallelism().set(nproc=8)

## Spacetime manifold

We declare the Kerr spacetime (or more precisely the part of it covered by Boyer-Lindquist coordinates) as a 4-dimensional Lorentzian manifold $\mathcal{M}$:

In [4]:
M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


We then introduce the standard **Boyer-Lindquist coordinates** $(t,r,\theta,\phi)$ as a chart `BL` (for *Boyer-Lindquist*) on $\mathcal{M}$, via the method `chart()`. The argument of the latter is a string
(delimited by `r"..."` because of the backslash symbols) expressing the coordinates names, their ranges (the default is $(-\infty,+\infty)$) and their LaTeX symbols:

In [5]:
BL.<t,r,th,ph> = M.chart(r"t r th:(0,pi):\theta ph:(0,2*pi):periodic:\phi") 
print(BL); BL

Chart (M, (t, r, th, ph))


Chart (M, (t, r, th, ph))

In [6]:
BL[0], BL[1]

(t, r)

## Metric tensor of the extremal Kerr spacetime

The mass parameter $m$ of the Kerr spacetime is declared as a symbolic variable:

In [7]:
m = var('m', domain='real')

For the extremal Kerr black hole, the spin parameter $a$ reaches its upper bound, namely $m$:

In [8]:
a = m

We get the (yet undefined) spacetime metric:

In [9]:
g = M.metric()

<p>The metric is set by its components in the coordinate frame associated with Boyer-Lindquist coordinates, which is the current manifold's default frame:</p>

In [10]:
rho2 = r^2 + (a*cos(th))^2
Delta = r^2 -2*m*r + a^2
g[0,0] = -(1-2*m*r/rho2)
g[0,3] = -2*a*m*r*sin(th)^2/rho2
g[1,1], g[2,2] = rho2/Delta, rho2
g[3,3] = (r^2+a^2+2*m*r*(a*sin(th))^2/rho2)*sin(th)^2
g.display()

g = (2*m*r/(m^2*cos(th)^2 + r^2) - 1) dt*dt - 2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) dt*dph + (m^2*cos(th)^2 + r^2)/(m^2 - 2*m*r + r^2) dr*dr + (m^2*cos(th)^2 + r^2) dth*dth - 2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) dph*dt + (2*m^3*r*sin(th)^2/(m^2*cos(th)^2 + r^2) + m^2 + r^2)*sin(th)^2 dph*dph

<p>A matrix view of the components with respect to the manifold's default vector frame:</p>

In [11]:
g[:]

[                                2*m*r/(m^2*cos(th)^2 + r^2) - 1                                                               0                                                               0                        -2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2)]
[                                                              0                       (m^2*cos(th)^2 + r^2)/(m^2 - 2*m*r + r^2)                                                               0                                                               0]
[                                                              0                                                               0                                             m^2*cos(th)^2 + r^2                                                               0]
[                       -2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2)                                                               0                                                               0 (2*m^3*r*sin(th)^2/(m^2*cos(th)^2 + r^2) + m^2 + r^2)*sin(th)^2]

<p>The list of the non-vanishing components:</p>

In [12]:
g.display_comp()

g_t,t = 2*m*r/(m^2*cos(th)^2 + r^2) - 1 
g_t,ph = -2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) 
g_r,r = (m^2*cos(th)^2 + r^2)/(m^2 - 2*m*r + r^2) 
g_th,th = m^2*cos(th)^2 + r^2 
g_ph,t = -2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) 
g_ph,ph = (2*m^3*r*sin(th)^2/(m^2*cos(th)^2 + r^2) + m^2 + r^2)*sin(th)^2

Let us check that we are dealing with a solution of the vacuum Einstein equation:

In [13]:
g.ricci().display()

Ric(g) = 0

## Near-horizon coordinates

Let us introduce the chart `NH` of the near-horizon coordinates $(\bar{t}, \bar{r}, \theta, \bar{\phi})$:

In [14]:
NH.<tb,rb,th,phb> = M.chart(r"tb:\bar{t} rb:\bar{r} th:(0,pi):\theta phb:(0,2*pi):periodic:\bar{\phi}") 
print(NH)
NH

Chart (M, (tb, rb, th, phb))


Chart (M, (tb, rb, th, phb))

In [15]:
M.atlas()

[Chart (M, (t, r, th, ph)), Chart (M, (tb, rb, th, phb))]

Following J. Bardeen and G. T. Horowitz, 
[Phys. Rev. D **60**, 104030 (1999)](https://doi.org/10.1103/PhysRevD.60.104030), the near-horizon coordinates $(\bar{t}, \bar{r}, \theta, \bar{\phi})$ are related to the Boyer-Lindquist coordinates by
$$\bar{t} = \epsilon t, \quad \bar{r} = \frac{r-m}{\epsilon},\quad \theta = \theta, 
\quad\bar{\phi} = \phi - \frac{t}{2m},$$
where $\epsilon$ is a constant parameter.
The horizon of the extremal Kerr black hole is located at $r=m$, which corresponds to $\bar{r}=0$.

We implement the above relations as a transition map from the chart `BL` to the chart `NH`:

In [16]:
eps = var('eps', latex_name=r'\epsilon')
BL_to_NH = BL.transition_map(NH, [eps*t, (r-m)/eps, th, ph - t/(2*m)])
BL_to_NH.display()

tb = eps*t
rb = -(m - r)/eps
th = th
phb = ph - 1/2*t/m

The inverse relation is

In [17]:
BL_to_NH.inverse().display()

t = tb/eps
r = eps*rb + m
th = th
ph = 1/2*(2*eps*m*phb + tb)/(eps*m)

Note that we have

In [18]:
BL_to_NH.jacobian()

[   eps      0      0      0]
[     0  1/eps      0      0]
[     0      0      1      0]
[-1/2/m      0      0      1]

and

In [19]:
BL_to_NH.jacobian_det()

1

The metric components with respect the coordinates 
$(\bar{t}, \bar{r}, \theta, \bar{\phi})$ are computed by passing the 
chart `NH` to the method `display()`:

In [20]:
M.default_chart()

Chart (M, (t, r, th, ph))

In [21]:
g.display()

g = (2*m*r/(m^2*cos(th)^2 + r^2) - 1) dt*dt - 2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) dt*dph + (m^2*cos(th)^2 + r^2)/(m^2 - 2*m*r + r^2) dr*dr + (m^2*cos(th)^2 + r^2) dth*dth - 2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) dph*dt + (2*m^3*r*sin(th)^2/(m^2*cos(th)^2 + r^2) + m^2 + r^2)*sin(th)^2 dph*dph

In [22]:
g.display(NH)

g = -1/4*(m^2*rb^2*cos(th)^4 - eps^2*rb^4 - 4*eps*m*rb^3 - 3*m^2*rb^2 + (eps^2*rb^4 + 4*eps*m*rb^3 + 6*m^2*rb^2)*cos(th)^2)/(eps^2*m^2*rb^2 + m^4*cos(th)^2 + 2*eps*m^3*rb + m^4) dtb*dtb - 1/2*(eps*m^2*rb^2*sin(th)^4 - (eps^3*rb^4 + 4*eps^2*m*rb^3 + 8*eps*m^2*rb^2 + 4*m^3*rb)*sin(th)^2)/(eps^2*m*rb^2 + m^3*cos(th)^2 + 2*eps*m^2*rb + m^3) dtb*dphb + (eps^2*rb^2 + m^2*cos(th)^2 + 2*eps*m*rb + m^2)/rb^2 drb*drb + (eps^2*rb^2 + m^2*cos(th)^2 + 2*eps*m*rb + m^2) dth*dth - 1/2*(eps*m^2*rb^2*sin(th)^4 - (eps^3*rb^4 + 4*eps^2*m*rb^3 + 8*eps*m^2*rb^2 + 4*m^3*rb)*sin(th)^2)/(eps^2*m*rb^2 + m^3*cos(th)^2 + 2*eps*m^2*rb + m^3) dphb*dtb - (eps^2*m^2*rb^2*sin(th)^4 - (eps^4*rb^4 + 4*eps^3*m*rb^3 + 8*eps^2*m^2*rb^2 + 8*eps*m^3*rb + 4*m^4)*sin(th)^2)/(eps^2*rb^2 + m^2*cos(th)^2 + 2*eps*m*rb + m^2) dphb*dphb

From now on, we use the near-horizon coordinates as the default ones on the spacetime manifold:

In [23]:
M.set_default_chart(NH)
M.set_default_frame(NH.frame())

Hence `NH` becomes the default argument of `display()`:

In [24]:
g.display()

g = -1/4*(m^2*rb^2*cos(th)^4 - eps^2*rb^4 - 4*eps*m*rb^3 - 3*m^2*rb^2 + (eps^2*rb^4 + 4*eps*m*rb^3 + 6*m^2*rb^2)*cos(th)^2)/(eps^2*m^2*rb^2 + m^4*cos(th)^2 + 2*eps*m^3*rb + m^4) dtb*dtb - 1/2*(eps*m^2*rb^2*sin(th)^4 - (eps^3*rb^4 + 4*eps^2*m*rb^3 + 8*eps*m^2*rb^2 + 4*m^3*rb)*sin(th)^2)/(eps^2*m*rb^2 + m^3*cos(th)^2 + 2*eps*m^2*rb + m^3) dtb*dphb + (eps^2*rb^2 + m^2*cos(th)^2 + 2*eps*m*rb + m^2)/rb^2 drb*drb + (eps^2*rb^2 + m^2*cos(th)^2 + 2*eps*m*rb + m^2) dth*dth - 1/2*(eps*m^2*rb^2*sin(th)^4 - (eps^3*rb^4 + 4*eps^2*m*rb^3 + 8*eps*m^2*rb^2 + 4*m^3*rb)*sin(th)^2)/(eps^2*m*rb^2 + m^3*cos(th)^2 + 2*eps*m^2*rb + m^3) dphb*dtb - (eps^2*m^2*rb^2*sin(th)^4 - (eps^4*rb^4 + 4*eps^3*m*rb^3 + 8*eps^2*m^2*rb^2 + 8*eps*m^3*rb + 4*m^4)*sin(th)^2)/(eps^2*rb^2 + m^2*cos(th)^2 + 2*eps*m*rb + m^2) dphb*dphb

## The near-horizon metric $h$ as the limit $\epsilon\to 0$ of the Kerr metric $g$

Let us define the *near-horizon metric* as the metric $h$ on $\mathcal{M}$ that is the limit $\epsilon\to 0$ of the Kerr metric $g$. The limit is taken by asking for a series expansion of $g$ with respect to $\epsilon$ up to the 0-th order (i.e. keeping only $\epsilon^0$ terms). This is acheived via the method `truncate`:

In [25]:
h = M.lorentzian_metric('h')
h.set(g.truncate(eps, 0))
h.display()

h = -1/4*(rb^2*cos(th)^4 + 6*rb^2*cos(th)^2 - 3*rb^2)/(m^2*cos(th)^2 + m^2) dtb*dtb + 2*rb*sin(th)^2/(cos(th)^2 + 1) dtb*dphb + (m^2*cos(th)^2 + m^2)/rb^2 drb*drb + (m^2*cos(th)^2 + m^2) dth*dth + 2*rb*sin(th)^2/(cos(th)^2 + 1) dphb*dtb + 4*m^2*sin(th)^2/(cos(th)^2 + 1) dphb*dphb

In [26]:
h.display_comp()

h_tb,tb = -1/4*(rb^2*cos(th)^4 + 6*rb^2*cos(th)^2 - 3*rb^2)/(m^2*cos(th)^2 + m^2) 
h_tb,phb = 2*rb*sin(th)^2/(cos(th)^2 + 1) 
h_rb,rb = (m^2*cos(th)^2 + m^2)/rb^2 
h_th,th = m^2*cos(th)^2 + m^2 
h_phb,tb = 2*rb*sin(th)^2/(cos(th)^2 + 1) 
h_phb,phb = 4*m^2*sin(th)^2/(cos(th)^2 + 1)

The $\epsilon$-expansion of $g$ has been performed on all sets of components. Those in the Boyer-Lindquist frame are not valid since nothing in them depend explicitely on $\epsilon$. To have consistent sets of components, we simply delete these components:

In [27]:
del h._components[BL.frame()]

As a check, we compare with Eq. (2.6) of [Bardeen & Horowitz, Phys. Rev. D **60**, 104030 (1999)](https://doi.org/10.1103/PhysRevD.60.104030), which involves the bilinear form
$$ q = \frac{4m^2\sin^2\theta}{1+\cos^2\theta} 
\left( \mathrm{d}\bar{\phi} + \frac{\bar{r}}{2m^2} \mathrm{d}\bar{t} \right)^2$$
We construct $h$ by getting first the 1-forms $\mathrm{d}\bar{t}$ and
$\mathrm{d}\bar{\phi}$:

In [28]:
dtb = NH.coframe()[0]
dtb

1-form dtb on the 4-dimensional Lorentzian manifold M

In [29]:
dphb = NH.coframe()[3]
dphb

1-form dphb on the 4-dimensional Lorentzian manifold M

In [30]:
q = 4*m^2*sin(th)^2/(1+cos(th)^2) * (dphb + rb/(2*m^2)*dtb) * (dphb + rb/(2*m^2)*dtb)
q.display()

rb^2*sin(th)^2/(m^2*cos(th)^2 + m^2) dtb*dtb + 2*rb*sin(th)^2/(cos(th)^2 + 1) dtb*dphb + 2*rb*sin(th)^2/(cos(th)^2 + 1) dphb*dtb + 4*m^2*sin(th)^2/(cos(th)^2 + 1) dphb*dphb

We evaluate then $h-q$ to compare with Eq. (2.6) of Bardeen & Horowitz:

In [31]:
(h - q).display()

-1/4*(rb^2*cos(th)^2 + rb^2)/m^2 dtb*dtb + (m^2*cos(th)^2 + m^2)/rb^2 drb*drb + (m^2*cos(th)^2 + m^2) dth*dth

In [32]:
s = (h - q)/(1+cos(th)^2)
s.display()

-1/4*(rb^2*cos(th)^2 + rb^2)/((cos(th)^2 + 1)*m^2) dtb*dtb + (m^2*cos(th)^2 + m^2)/((cos(th)^2 + 1)*rb^2) drb*drb + (m^2*cos(th)^2 + m^2)/(cos(th)^2 + 1) dth*dth

In [33]:
for i in range(4):
    for j in range(i, 4):
        s[i,j].simplify()
s.display()

-1/4*rb^2/m^2 dtb*dtb + m^2/rb^2 drb*drb + m^2 dth*dth

The above result shows the complete agreement with Eq. (2.6) of Bardeen & Horowitz.

We note that the metric $h$ is not asymptotically flat. 
Besides, it is a solution of the vacuum Einstein equation:

In [34]:
h.ricci().display()

Ric(h) = 0

## Killing vectors of the near-horizon geometry

Let us first consider the vector field $\eta := \frac{\partial}{\partial\bar{\phi}}$:

In [35]:
eta = M.vector_field(0, 0, 0, 1, name='eta', latex_name=r'\eta')
eta.display()

eta = d/dphb

It is a Killing vector of the near-horizon metric, since the Lie derivative of $h$ along $\eta$ vanishes:

In [36]:
h.lie_derivative(eta).display()

0

This is not surprising since the components of $h$ are independent from $\bar{\phi}$.

Similarly, we can check that $\xi_1 := \frac{\partial}{\partial\bar{t}}$ is a Killing vector of $h$, reflecting the independence of the components of $h$ from $\bar{t}$:

In [37]:
xi1 = M.vector_field(1, 0, 0, 0, name='xi1', latex_name=r'\xi_{1}')
xi1.display()

xi1 = d/dtb

In [38]:
h.lie_derivative(xi1).display()

0

The above two Killing vectors correspond respectively to the **axisymmetry** and the **pseudo-stationarity** of the Kerr metric. A third symmetry, which is not present in the original Kerr metric, is the invariance under the **scaling** $(\bar{t},\bar{r})\mapsto (\alpha \bar{t}, \bar{r}/\alpha)$, as it is clear on the metric components in Out[21]. The corresponding Killing vector is 

In [39]:
xi2 = M.vector_field(tb, -rb, 0, 0, name='xi2', latex_name=r'\xi_{2}')
xi2.display()

xi2 = tb d/dtb - rb d/drb

In [40]:
h.lie_derivative(xi2).display()

0

Finally, a fourth Killing vector is

In [41]:
xi3 = M.vector_field(tb^2/2 + 2*m^4/rb^2, -tb*rb, 0, -2*m^2/rb,
                     name='xi3', latex_name=r'\xi_{3}')
xi3.display()

xi3 = (2*m^4/rb^2 + 1/2*tb^2) d/dtb - rb*tb d/drb - 2*m^2/rb d/dphb

In [42]:
h.lie_derivative(xi3).display()

0

## Symmetry group

We have four independent Killing vectors, $\eta$, $\xi_1$, $\xi_2$ and $\xi_3$, which implies that the symmetry group of the near-horizon geometry is a 4-dimensional Lie group $G$. Let us determine $G$ by investigating the **structure constants** of the basis 
$(\eta, \xi_1, \xi_2, \xi_3)$ of the Lie algebra of $G$. First of all, we notice that 
$\eta$ commutes with the other Killing vectors:

In [43]:
for xi in [xi1, xi2, xi3]:
    show(eta.bracket(xi).display())

[eta,xi1] = 0

[eta,xi2] = 0

[eta,xi3] = 0

Since $\eta$ generates the rotation group $\mathrm{SO}(2)=\mathrm{U}(1)$, we may write that $G = \mathrm{U}(1)\times G_3$, where $G_3$
is a 3-dimensional Lie group, whose generators are $(\xi_1, \xi_2, \xi_3)$. Let us determine the structure constants of these three vectors. We have

In [44]:
xi1.bracket(xi2).display()

[xi1,xi2] = d/dtb

In [45]:
xi1.bracket(xi2) == xi1

True

In [46]:
xi1.bracket(xi3).display()

[xi1,xi3] = tb d/dtb - rb d/drb

In [47]:
xi1.bracket(xi3) == xi2

True

In [48]:
xi2.bracket(xi3).display()

[xi2,xi3] = 1/2*(4*m^4 + rb^2*tb^2)/rb^2 d/dtb - rb*tb d/drb - 2*m^2/rb d/dphb

In [49]:
xi2.bracket(xi3) == xi3

True

To summarize, we have

In [50]:
all([xi1.bracket(xi2) == xi1,
     xi1.bracket(xi3) == xi2,
     xi2.bracket(xi3) == xi3])

True

To recognize a standard Lie algebra, let us perform a slight change of basis:

In [51]:
vE = -sqrt(2)*xi3
vF = sqrt(2)*xi1
vH = 2*xi2

We have then the following commutation relations:

In [52]:
all([vE.bracket(vF) == vH,
     vH.bracket(vE) == 2*vE,
     vH.bracket(vF) == -2*vF])

True

We recognize the Lie algebra $\mathfrak{sl}(2, \mathbb{R})$. Indeed, we have, using the representation of $\mathfrak{sl}(2, \mathbb{R})$ by traceless $2\times 2$ matrices:

In [53]:
sl2 = lie_algebras.sl(QQ, 2, representation='matrix')
E,F,H = sl2.gens()

In [54]:
E

[0 1]
[0 0]

In [55]:
F

[0 0]
[1 0]

In [56]:
H

[ 1  0]
[ 0 -1]

In [57]:
all([E.bracket(F) == H,
     H.bracket(E) == 2*E,
     H.bracket(F) == -2*F])

True

Hence, we have $$\mathrm{Lie}(G_3) = \mathfrak{sl}(2, \mathbb{R}).$$

At this stage, $G_3$ could be $\mathrm{SL}(2, \mathbb{R})$, $\mathrm{PSL}(2, \mathbb{R})$
or $\overline{\mathrm{SL}(2, \mathbb{R})}$ (the universal covering group of $\mathrm{SL}(2, \mathbb{R})$). One can show that actually 
$G_3 = \mathrm{SL}(2, \mathbb{R})$.
We conclude that the full isometry group of the near-horizon geometry is 
$$ G = \mathrm{U}(1) \times \mathrm{SL}(2, \mathbb{R}).$$

## Expression of the near-horizon Killing vectors in the Boyer-Lindquist basis

We have:

In [58]:
eta.display(BL)

eta = d/dph

In [59]:
xi1.display(BL)

xi1 = 1/eps d/dt + 1/2/(eps*m) d/dph

Given that for an extremal Kerr black hole $\Omega_H = 1/(2m)$, we recognize that 
$$ \xi_1 = \epsilon^{-1} \chi $$
where $\chi := \frac{\partial}{\partial t} + \Omega_H \frac{\partial}{\partial \phi}$ is the
Killing vector that is tangent to the horizon null generators on $\mathcal{H}$, or equivalently that is the null normal of the Killing horizon $\mathcal{H}$. 

In [60]:
xi2.display(BL)

xi2 = t d/dt + (m - r) d/dr + 1/2*t/m d/dph

In [61]:
xi3.display(BL)

xi3 = 1/2*(4*eps*m^4 + (eps*m^2 - 2*eps*m*r + eps*r^2)*t^2)/(m^2 - 2*m*r + r^2) d/dt + (eps*m - eps*r)*t d/dr + 1/4*(12*eps*m^4 - 8*eps*m^3*r + (eps*m^2 - 2*eps*m*r + eps*r^2)*t^2)/(m^3 - 2*m^2*r + m*r^2) d/dph

In [62]:
for i in M.irange():
    xi3[BL.frame(), i, BL].factor()
xi3.display(BL)

xi3 = 1/2*(4*m^4 + m^2*t^2 - 2*m*r*t^2 + r^2*t^2)*eps/(m - r)^2 d/dt + eps*(m - r)*t d/dr + 1/4*(12*m^4 - 8*m^3*r + m^2*t^2 - 2*m*r*t^2 + r^2*t^2)*eps/((m - r)^2*m) d/dph

### The Killing operator of $g$ applied to the near-horizon Killing vectors

In this part, we set the Boyer-Lindquist chart and frame back to the defaults ones:

In [63]:
M.set_default_chart(BL)
M.set_default_frame(BL.frame())

The Levi-Civita connection $\nabla$ of $g$:

In [64]:
nabla = g.connection()

The Killing form $\mathrm{K}v$ of a vector field $v$ is defined by
$$(\mathrm{K}v)_{ab} := \nabla_a v_b + \nabla_b v_a,$$
where $v_a := g_{ab} v^b$. With SageMath, this becomes

In [65]:
def killing_form(v):
    Kv = 2*nabla(v.down(g)).symmetrize()
    if v._name is not None:
        Kv.set_name('K' + v._name, latex_name=r'\mathrm{K}' + str(latex(v)))
    return Kv

For some reason (to be investigated...), what follows does not work with multiprocessing:

In [66]:
Parallelism().set(nproc=1)

The Killing form of $\eta$ is identically zero:

In [67]:
Keta = killing_form(eta)
print(Keta)
Keta.display()

Field of symmetric bilinear forms Keta on the 4-dimensional Lorentzian manifold M


Keta = 0

We recover the fact that $\eta$ is a Killing vector of $g$. Similarly, we have 

In [68]:
Kxi1 = killing_form(xi1)
print(Kxi1)
Kxi1.display()

Field of symmetric bilinear forms Kxi1 on the 4-dimensional Lorentzian manifold M


Kxi1 = 0

We recover the fact that $\xi_1 = \epsilon^{-1}\chi$ is a Killing vector of $g$.

But $\xi_2$ and $\xi_3$ are not Killing vectors of $g$:

In [69]:
Kxi2 = killing_form(xi2)
print(Kxi2)
Kxi2.display()

Field of symmetric bilinear forms Kxi2 on the 4-dimensional Lorentzian manifold M


Kxi2 = -2*((m^4 - m^3*r)*cos(th)^4 + m^2*r^2 - 2*m*r^3 + r^4 - (m^4 - 2*m^2*r^2 + m*r^3)*cos(th)^2)/(m^4*cos(th)^4 + 2*m^2*r^2*cos(th)^2 + r^4) dt*dt + 1/2*((m^6 - 2*m^5*r + m^4*r^2)*sin(th)^6 + 2*(m^6 + m^5*r - 2*m^4*r^2 + m^3*r^3 - m^2*r^4)*sin(th)^4 - (3*m^6 - 7*m^4*r^2 + 8*m^3*r^3 - 3*m^2*r^4 - r^6)*sin(th)^2)/(m^5*cos(th)^4 + 2*m^3*r^2*cos(th)^2 + m*r^4) dt*dph + 2*r/(m - r) dr*dr + (2*m*r - 2*r^2) dth*dth + 1/2*((m^6 - 2*m^5*r + m^4*r^2)*sin(th)^6 + 2*(m^6 + m^5*r - 2*m^4*r^2 + m^3*r^3 - m^2*r^4)*sin(th)^4 - (3*m^6 - 7*m^4*r^2 + 8*m^3*r^3 - 3*m^2*r^4 - r^6)*sin(th)^2)/(m^5*cos(th)^4 + 2*m^3*r^2*cos(th)^2 + m*r^4) dph*dt - 2*((m^4*r^2 - m^3*r^3 - (m^6 - m^5*r)*cos(th)^2)*sin(th)^4 - (m*r^5 - r^6 + (m^5*r - m^4*r^2)*cos(th)^4 + 2*(m^3*r^3 - m^2*r^4)*cos(th)^2)*sin(th)^2)/(m^4*cos(th)^4 + 2*m^2*r^2*cos(th)^2 + r^4) dph*dph

In [70]:
Kxi3 = killing_form(xi3)
print(Kxi3)
Kxi3.display()

Field of symmetric bilinear forms Kxi3 on the 4-dimensional Lorentzian manifold M


Kxi3 = -2*((eps*m^4 - eps*m^3*r)*t*cos(th)^4 - (eps*m^4 - 2*eps*m^2*r^2 + eps*m*r^3)*t*cos(th)^2 + (eps*m^2*r^2 - 2*eps*m*r^3 + eps*r^4)*t)/(m^4*cos(th)^4 + 2*m^2*r^2*cos(th)^2 + r^4) dt*dt + (eps*m^4*cos(th)^4 + eps*r^4 - 2*(2*eps*m^4 - eps*m^2*r^2)*cos(th)^2)/(m*r^2 - r^3 + (m^3 - m^2*r)*cos(th)^2) dt*dr + 1/2*((eps*m^6 - 2*eps*m^5*r + eps*m^4*r^2)*t*sin(th)^6 + 2*(eps*m^6 + eps*m^5*r - 2*eps*m^4*r^2 + eps*m^3*r^3 - eps*m^2*r^4)*t*sin(th)^4 - (3*eps*m^6 - 7*eps*m^4*r^2 + 8*eps*m^3*r^3 - 3*eps*m^2*r^4 - eps*r^6)*t*sin(th)^2)/(m^5*cos(th)^4 + 2*m^3*r^2*cos(th)^2 + m*r^4) dt*dph + (eps*m^4*cos(th)^4 + eps*r^4 - 2*(2*eps*m^4 - eps*m^2*r^2)*cos(th)^2)/(m*r^2 - r^3 + (m^3 - m^2*r)*cos(th)^2) dr*dt + 2*eps*r*t/(m - r) dr*dr - 2*((2*eps*m^5 - eps*m^4*r)*sin(th)^4 - (2*eps*m^5 - eps*m^4*r - eps*m^2*r^3)*sin(th)^2)/(m*r^2 - r^3 + (m^3 - m^2*r)*cos(th)^2) dr*dph + 2*(eps*m*r - eps*r^2)*t dth*dth + 1/2*((eps*m^6 - 2*eps*m^5*r + eps*m^4*r^2)*t*sin(th)^6 + 2*(eps*m^6 + eps*m^5*r - 2*eps*m^4*r^2 + eps*m^3*r^3 - eps*m^2*r^4)*t*sin(th)^4 - (3*eps*m^6 - 7*eps*m^4*r^2 + 8*eps*m^3*r^3 - 3*eps*m^2*r^4 - eps*r^6)*t*sin(th)^2)/(m^5*cos(th)^4 + 2*m^3*r^2*cos(th)^2 + m*r^4) dph*dt - 2*((2*eps*m^5 - eps*m^4*r)*sin(th)^4 - (2*eps*m^5 - eps*m^4*r - eps*m^2*r^3)*sin(th)^2)/(m*r^2 - r^3 + (m^3 - m^2*r)*cos(th)^2) dph*dr + 2*(((eps*m^6 - eps*m^5*r)*t*cos(th)^2 - (eps*m^4*r^2 - eps*m^3*r^3)*t)*sin(th)^4 + ((eps*m^5*r - eps*m^4*r^2)*t*cos(th)^4 + 2*(eps*m^3*r^3 - eps*m^2*r^4)*t*cos(th)^2 + (eps*m*r^5 - eps*r^6)*t)*sin(th)^2)/(m^4*cos(th)^4 + 2*m^2*r^2*cos(th)^2 + r^4) dph*dph

In [71]:
Kxi3.truncate(eps, 0).display()

0